In [12]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from google.cloud import language_v1
import boto3
from dotenv import load_dotenv
import os

load_dotenv()
IBM_API_KEY = os.getenv("IBM_API_KEY")
IBM_URL = os.getenv("IBM_URL")

GOOGLE_API_CREDENTIALS_PATH = os.getenv("GOOGLE_API_CREDENTIALS_PATH")

AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")
AWS_REGION = os.getenv("AWS_REGION")

In [4]:
# IBM Watson NLU Sentiment Analysis
def analyze_sentiment_ibm(text):
    authenticator = IAMAuthenticator(IBM_API_KEY)
    nlu = NaturalLanguageUnderstandingV1(
        version="2021-08-01",
        authenticator=authenticator
    )
    nlu.set_service_url(IBM_URL)

    response = nlu.analyze(
        text=text,
        features=Features(sentiment=SentimentOptions())
    ).get_result()

    sentiment = response['sentiment']['document']['label']
    return sentiment

# Google Cloud NLP Sentiment Analysis
def analyze_sentiment_google(text):
    client = language_v1.LanguageServiceClient.from_service_account_json(GOOGLE_API_CREDENTIALS_PATH)

    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    response = client.analyze_sentiment(document=document)

    score = response.document_sentiment.score
    if score > 0.25:
        return "positive"
    elif score < -0.25:
        return "negative"
    else:
        return "neutral"

# AWS Comprehend Sentiment Analysis
def analyze_sentiment_aws(text):
    client = boto3.client(
        "comprehend",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )

    response = client.detect_sentiment(
        Text=text,
        LanguageCode="en"
    )

    sentiment = response['Sentiment']
    return sentiment.lower()  # Convert to lowercase for consistency

In [5]:
# Evaluate Accuracy
def evaluate_accuracy(predictions, ground_truth):
    correct = sum(1 for pred, truth in zip(predictions, ground_truth) if pred == truth)
    accuracy = (correct / len(ground_truth)) * 100
    return accuracy

In [6]:
import random

# Simulated dataset of reviews
reviews = []
ground_truths = []

# Lists of predefined phrases for positive, neutral, and negative reviews
positive_phrases = [
    "This product is amazing and exceeded my expectations.",
    "I absolutely love it! Highly recommended.",
    "Great value for money. Worth every penny.",
    "Fantastic quality and superb performance.",
    "The service was excellent and very quick.",
    "I'm very happy with this purchase.",
    "Perfect for my needs. Will buy again.",
    "Great design and very user-friendly.",
    "I'm extremely satisfied with the results.",
    "This is the best thing I've ever bought."
]

neutral_phrases = [
    "The product is okay but not outstanding.",
    "It works as expected, nothing special.",
    "Average experience, neither good nor bad.",
    "The features are decent but could be better.",
    "It's fine, but I've seen better.",
    "Shipping was delayed but the product is usable.",
    "It's acceptable for the price, but not great.",
    "Meh, it's just okay.",
    "Not bad, but not great either.",
    "I have mixed feelings about this product."
]

negative_phrases = [
    "Terrible quality, completely disappointed.",
    "Not worth the money. Do not buy this.",
    "The product broke after one use.",
    "Horrible experience. Never again.",
    "The worst purchase I've ever made.",
    "It's overpriced and does not work as described.",
    "Customer service was unhelpful and rude.",
    "The design is flawed and poorly executed.",
    "The item arrived damaged and unusable.",
    "This product is a complete waste of money."
]

# Generate 500 reviews with balanced sentiment
for _ in range(167):  # Approximately 1/3 positive, neutral, negative
    reviews.append(random.choice(positive_phrases))
    ground_truths.append("positive")

    reviews.append(random.choice(neutral_phrases))
    ground_truths.append("neutral")

    reviews.append(random.choice(negative_phrases))
    ground_truths.append("negative")

# Ensure exactly 500 reviews
reviews.append(random.choice(positive_phrases))
ground_truths.append("positive")

# Shuffle the dataset to mix sentiments
combined = list(zip(reviews, ground_truths))
random.shuffle(combined)
reviews, ground_truths = zip(*combined)

# Convert to lists
reviews = list(reviews)
ground_truths = list(ground_truths)


In [7]:
ibm_predictions = []
for review in reviews:
    ibm_predictions.append(analyze_sentiment_ibm(review))

# Calculate accuracy for each tool
ibm_accuracy = evaluate_accuracy(ibm_predictions, ground_truths)

In [8]:
google_predictions = []
for review in reviews:
    google_predictions.append(analyze_sentiment_google(review))

# Calculate accuracy for each tool
google_accuracy = evaluate_accuracy(google_predictions, ground_truths)

In [9]:
aws_predictions = []
for review in reviews:
    aws_predictions.append(analyze_sentiment_aws(review))

# Calculate accuracy for each tool
aws_accuracy = evaluate_accuracy(aws_predictions, ground_truths)

In [10]:
print("\nSentiment Analysis Results:")
print(f"IBM Watson NLU Accuracy: {ibm_accuracy:.2f}%")
print(f"Google Cloud NLP Accuracy: {google_accuracy:.2f}%")
print(f"AWS Comprehend Accuracy: {aws_accuracy:.2f}%")


Sentiment Analysis Results:
IBM Watson NLU Accuracy: 68.53%
Google Cloud NLP Accuracy: 94.62%
AWS Comprehend Accuracy: 71.71%
